In [3]:
import requests                             ##Import of my libraries. I thought I might need to flip pages, but actually the url worked well with f'strings.
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


In [2]:
num_pages = 86      ## I just hard-coded this to save time.
all_scrapes = []    ## List to hold all my dictionary rows which will be appended later.


In [4]:
#loop through elements on the page and get country, date, and subtext

all_scrapes = []

for page in range(num_pages):                  ## Loop through each page
    print(f'Scraping page number:{page}')      
    url = f'https://www.dsca.mil/press-media/major-arms-sales?page={page}'  ## F string to move through page numbers, page just goes up all the way to 86.
    print(url)                                 ## Tells me where I am
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    
    #put all items on page into a list
    page_content = soup.find_all("div", class_="col-md-4 col-sm-6 views-row")  ##Each of these classes is one 'square' on the page.
    for sale in page_content:
        
        header = sale.find('h2').getText()  ## Pulls in the bold text on the top of each square.


        ## Data cleaning. This site is terrible and interchanges em dashes and hyphens.
        ## Here I replace the em dash with hyphens to keep it consistent. Then I split on the first hyphen.
        
        header = header.replace('–','-')
        header = header.split('-',1)

        country_name = header[0]  ## The first item in the resulting list is always going to be the country, thank god.
        
        try:
            weapon = header[1]    ## Sometimes the data are incomplete, so I couched everything in Try functions to avoid errors.
                                  ## This gets the second item, which is always the weapon itself.
        except:
            weapon = "null"       ## If for some reason it's not there, write null, instead of breaking the whole scrape.
        
        try:
            subhed = sale.find_all('p')[-1].getText()
            
        except:
            subhed = "null"
        
        try:
            date = subhed.replace('–','-')  ## Same cleaning as above.
            date = date.split('-',1)        ## Separate the date from the big text to the right. Returns two strings with ['Washington, October 3, 2020','text']
            date = date[0].split(',')[-2:]  ## Splits the first item in the list on the commas. Gets the date by pulling the last two items. I.e. October 3 and 2020
            date = date[-2] + ',' + date[-1]## Make a recognizal datetime string.   
            
                                            ## This part of the scrape could use some fine-tuning. It works until the website implements a style change by removing hyphens. 
                                            ## Then it returns messy data.
        except:
            date = "null"
            
        try:
            text = subhed.split(' - ')[1]
        except:
            text = subhed.split(' - ')
            
        row = {"country":country_name, "weapon":weapon, "date": date, "text":text}
        all_scrapes.append(row)
        
data=pd.DataFrame(all_scrapes)
data.tail(50)
data.to_csv("arms_data.csv")

Scraping page number:0
https://www.dsca.mil/press-media/major-arms-sales?page=0
Scraping page number:1
https://www.dsca.mil/press-media/major-arms-sales?page=1
Scraping page number:2
https://www.dsca.mil/press-media/major-arms-sales?page=2
Scraping page number:3
https://www.dsca.mil/press-media/major-arms-sales?page=3
Scraping page number:4
https://www.dsca.mil/press-media/major-arms-sales?page=4
Scraping page number:5
https://www.dsca.mil/press-media/major-arms-sales?page=5
Scraping page number:6
https://www.dsca.mil/press-media/major-arms-sales?page=6
Scraping page number:7
https://www.dsca.mil/press-media/major-arms-sales?page=7
Scraping page number:8
https://www.dsca.mil/press-media/major-arms-sales?page=8
Scraping page number:9
https://www.dsca.mil/press-media/major-arms-sales?page=9
Scraping page number:10
https://www.dsca.mil/press-media/major-arms-sales?page=10
Scraping page number:11
https://www.dsca.mil/press-media/major-arms-sales?page=11
Scraping page number:12
https://www.

In [97]:
data.to_csv("arms_data.csv", index=False)